# SuperAI Season 4 - Level Individual Hackathon - Sleep Stages Classification <br> Autoencoder

## Load data from database

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from tqdm.notebook import tqdm

In [4]:
with open ('./database/arrays.npy' , 'rb') as file :

    arrays = np.load(file)

with open ('./database/labels.npy' , 'rb') as file :

    labels = np.load(file)

In [5]:
print(arrays.shape , labels.shape)

(66607, 15360) (66607,)


In [6]:
id2label = {
    0 : 'W' ,
    1 : 'N' ,
    2 : 'R'
}
label2id = {
    'W' : 0 ,
    'N' : 1 ,
    'R' : 2
}

## Sleep Stages Dataset

In [8]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader , random_split
from torch.utils.data import Dataset
import torchmetrics
import lightning as L
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint

In [9]:
class sleep_stages_dataset (Dataset) :

    def __init__ (self , train_path , label_path , transform = None , target_transform = None) :

        self.train_path = train_path
        self.label_path = label_path

        with open (train_path , 'rb') as file :

            self.train_arrays = np.load(file)

        with open (label_path , 'rb') as file :

            self.label_arrays = np.load(file)

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) :

        return self.train_arrays.shape[0]

    def __getitem__ (self , idx) :

        data  = self.train_arrays[idx]
        label = self.label_arrays[idx]

        if self.transform :

            data = self.transform(data)

        if self.target_transform :

            label = self.target_transform(int(label))

        return data , label

In [10]:
class OneHotEncodeTransform:

    def __init__(self, num_classes):

        self.num_classes = num_classes

    def __call__(self, target):

        return torch.nn.functional.one_hot(torch.tensor(target), num_classes=self.num_classes ).type(torch.float)

In [11]:
transform = torch.Tensor
target_transform = OneHotEncodeTransform(num_classes = 3)

In [12]:
train_path = './database/arrays.npy'
label_path = './database/labels.npy'

In [13]:
train_dataset = sleep_stages_dataset(train_path = train_path , label_path = label_path , transform = transform , target_transform = target_transform)

In [14]:
# Define the size of each split
train_size = int(0.8 * len(train_dataset))
validation_size = len(train_dataset) - train_size

# Split the dataset
train_dataset , validation_dataset = random_split(train_dataset, [train_size , validation_size])

train_loader = DataLoader(train_dataset , batch_size = 256 , shuffle = True , num_workers = 11 )
validation_loader = DataLoader(validation_dataset , batch_size = 256 , shuffle = False , num_workers = 11)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
class encoder(nn.Module):

    def __init__(self , in_features , out_features):

        super().__init__()

        self.fc1 = nn.Linear(in_features, 7680)
        self.fc2 = nn.Linear(7680, 3840)
        self.fc3 = nn.Linear(3840, 1920)
        self.fc4 = nn.Linear(1920, 960)
        self.fc5 = nn.Linear(960, 480)
        self.fc6 = nn.Linear(480, out_features)

        self.batch_norm1 = nn.BatchNorm1d(7680)
        self.batch_norm2 = nn.BatchNorm1d(3840)
        self.batch_norm3 = nn.BatchNorm1d(1920)
        self.batch_norm4 = nn.BatchNorm1d(960)
        self.batch_norm5 = nn.BatchNorm1d(480)
        self.batch_norm6 = nn.BatchNorm1d(out_features)

        self.dropout = nn.Dropout(p = 0.2)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout(x)

        x = F.relu(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.dropout(x)

        x = F.relu(self.fc4(x))
        x = self.batch_norm4(x)
        x = self.dropout(x)

        x = F.relu(self.fc5(x))
        x = self.batch_norm5(x)
        x = self.dropout(x)

        x = F.relu(self.fc6(x))
        x = self.batch_norm6(x)
        x = self.dropout(x)

        return x

In [35]:
class decoder(nn.Module):

    def __init__(self , in_features , out_features):

        super().__init__()

        self.fc1 = nn.Linear(in_features, 480)
        self.fc2 = nn.Linear(480, 960)
        self.fc3 = nn.Linear(960, 1920)
        self.fc4 = nn.Linear(1920, 3840)
        self.fc5 = nn.Linear(3840, 7680)
        self.fc6 = nn.Linear(7680, out_features)

        self.batch_norm1 = nn.BatchNorm1d(480)
        self.batch_norm2 = nn.BatchNorm1d(960)
        self.batch_norm3 = nn.BatchNorm1d(1920)
        self.batch_norm4 = nn.BatchNorm1d(3840)
        self.batch_norm5 = nn.BatchNorm1d(7680)
        self.batch_norm6 = nn.BatchNorm1d(out_features)

        self.dropout = nn.Dropout(p = 0.2)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout(x)

        x = F.relu(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.dropout(x)

        x = F.relu(self.fc4(x))
        x = self.batch_norm4(x)
        x = self.dropout(x)

        x = F.relu(self.fc5(x))
        x = self.batch_norm5(x)
        x = self.dropout(x)

        x = F.relu(self.fc6(x))
        x = self.batch_norm6(x)
        x = self.dropout(x)

        return x

In [36]:
class autoencoder(L.LightningModule):

    def __init__(self , num_features):

        super().__init__()

        self.encoder = encoder(num_features , 240)
        self.decoder = decoder(240 , num_features)

    def forward (self , x) :

        x = self.encoder(x)
        x = self.decoder(x)

        return x

    def training_step(self, batch, batch_idx):

        x , y = batch

        y_hat = self(x.to(device))

        loss = F.cross_entropy(y_hat.squeeze() , x.to(device))

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):

        x , y = batch

        y_hat = self(x)

        val_loss = F.cross_entropy(y_hat.squeeze(), x.to(device))

        self.log('val_loss', val_loss, on_step = True, on_epoch = True, prog_bar = True)

    def configure_optimizers(self):

        optimizer = torch.optim.AdamW(self.parameters(), lr = 1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma=0.1)

        return [optimizer], [scheduler]

In [37]:
sleep_stages_autoencoder_model = autoencoder(num_features = 15360)

In [38]:
sleep_stages_autoencoder_model.to(device)

autoencoder(
  (encoder): encoder(
    (fc1): Linear(in_features=15360, out_features=7680, bias=True)
    (fc2): Linear(in_features=7680, out_features=3840, bias=True)
    (fc3): Linear(in_features=3840, out_features=1920, bias=True)
    (fc4): Linear(in_features=1920, out_features=960, bias=True)
    (fc5): Linear(in_features=960, out_features=480, bias=True)
    (fc6): Linear(in_features=480, out_features=240, bias=True)
    (batch_norm1): BatchNorm1d(7680, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm2): BatchNorm1d(3840, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm3): BatchNorm1d(1920, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm4): BatchNorm1d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm5): BatchNorm1d(480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm6): BatchNorm1d(240, eps=1e-05, momentum=0.1, affine=Tr

In [39]:
checkpoint_callback = ModelCheckpoint(dirpath='./models'  , filename='autoencoder_{epoch}-{val_loss:.2f}')

In [40]:
trainer = L.Trainer(max_epochs = 100 , callbacks=[checkpoint_callback])
trainer.fit(model = sleep_stages_autoencoder_model, train_dataloaders = train_loader, val_dataloaders = validation_loader )

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | encoder | encoder | 157 M  | train
1 | decoder | decoder | 157 M  | train
--------------------------------------------
314 M     Trainable params
0         Non-trainable params
314 M     Total params
1,258.528 Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | encoder | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

ResourceExhaustedError: /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Individual_Hackathon/Hack_Sleep_Stages_Classification/lightning_logs/version_5/events.out.tfevents.1718880383.0a982e20c433.275.2; No space left on device